## sub-task 1 제출 스크립트
ETRI FASHION-HOW Season 5 task1 제출코드입니다.
#### 주의: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.
- 작성하신 추론용 코드(예: test.py)를 본 스크립트 내의 etri_task1_submit() 함수로 작동되게끔 삽입하는 것으로 결과 제출을 수행할 수 있습니다.
코드는 크게 4가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 설치
2. etri_task1_submit() 함수 편집 (추론 스크립트)
3. submit() 함수로 wrapping
4. if __name__ == "__main__" 조건문을 이용한 제출 수행

※ 가능하면 제출시에는 포함되어 있는 train data를 폴더에서 제외하고 제출하시는 편이 좋습니다.
- 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 시간 감소

### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

### 2. etri_task1_submit() 함수 편집 (추론 스크립트)
#### etri_task1_submit() 편집 시 주의사항

1. 아래 etri_task1_submit() 함수 내에 전체 추론 실행 코드를 삽입하고 결과를 dataframe으로 return하게끔 구성
   - Baseline이 아닌 다른 모델을 사용하는 경우에도 동일
2. 함수 내에서는 import * 가 적용되지 않으므로 필요한 import object를 직접 입력
   - 반드시 함수 내에서 import가 이루어져야 합니다.
3. argparse 사용시 args, _ = parser.parse_known_args()로 인자 지정
   args = parser.parse_args()는 jupyter에서 오류가 발생합니다!!!
4. 모델 내부의 경로는 ./ 으로 경로를 지정합니다. (예: weight 파일 경로 = ./model/...)
5. 데이터는 **/aif/Dataset/** 경로 아래에 있습니다. (코드 내용 참조) 
6. return할 결과물과 양식에 유의합니다.

In [1]:
def etri_task1_submit():
    from dataset import ClassificationDataset
    from model import EVATiny

    import pandas as pd
    import numpy as np
    from augmentation import inference_transform
    import torch
    import torch.utils.data
    import torch.utils.data.distributed

    from tqdm import tqdm

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    net = EVATiny()
    net.load_state_dict(torch.load("./model/Best_EVATiny.pth", map_location=DEVICE))
    net.to(DEVICE)
    net.eval()
    transforms = inference_transform()

    df = pd.read_csv("/aif/Dataset/Fashion-How24_sub2_test.csv")  # 제출 시 데이터 경로 준수. /aif/ 아래에 있습니다.
    val_dataset = ClassificationDataset(root="/aif/Dataset/test/", df=df, transform=transforms, crop=False)

    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0)

    pred_list = np.array([])

    with torch.inference_mode():
        for _, (image, _) in tqdm(enumerate(val_dataloader)):
            image = image.to(DEVICE)
            out = net(image)
            _, indx = out.max(1)
            pred_list = np.concatenate([pred_list, indx.cpu()], axis=0)
            
    out = pd.DataFrame({'image_name':df['image_name'],'color':pred_list})

    return out  # 반드시 추론결과를 return

### 3. submit() 함수로 wrapping
#### 반드시 아래와 같이 submit() 이라는 함수로 위에 정의된 etri_task1_submit 함수를 wrapping해야 합니다.

In [2]:
def submit():
    return etri_task1_submit

### 4. if name == "main" 조건문을 이용한 제출 수행
#### 아래와 같이 if __name__ == "__main__" 구문 내에서 제출 함수가 실행되게끔 합니다.
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 각팀 팀장분에게 메일로 전달된 안내네서 task별로 확인하실 수 있습니다.

In [3]:
import aifactory.score as aif
import time
t = time.time()
if __name__ == "__main__":
    #-----------------------------------------------------#
    aif.submit(model_name="task2",           
               key="c2af3f08-221f-46b9-8ebe-065956419852",                            
               func=submit                                
               )
    #-----------------------------------------------------#
    print(time.time() - t)

file : task.py
python
파일 전송 완료
score = 0.86( 
409.7200219631195
